# Exercício de Modalagem e Simulação Computacional 1

Esse exercício consiste na análise de um sistema massa/mola/amortecedor com um grau de liberdade
através de integração numérica de sua EDO, variando-se as condições iniciais.

## Análise física do sistema

![](./Massa_mola_amortecedor.png)

O sistema, mostrado na imagem acima, apresenta uma mola, um amortecedor e uma força externa, sendo o atrito e a resistência do ar desprezados.
Devido ao vínculo com o chão e à ausência de forças externas na vertical, o bloco não se movimenta nesta direção.

Portanto, só há um grau de liberdade: na direção x, na qual as forças atuantes são as seguintes:

\begin{align}
&\mathbf{F} = F(t)\mathbf{\hat{i}} \\
&\mathbf{F_{mola}} = -kx\mathbf{\hat{i}} \\
&\mathbf{F_{amort}} = -c\dot{x}\mathbf{\hat{i}}
\end{align}

O que nos dá a resultante

$$\mathbf{R} = \mathbf{F + F_{mola} + F_{amort}}$$

Pela 2a Lei de Newton, temos:

$$\mathbf{R} = m\mathbf{\ddot{x}}$$

Como só temos movimento em x, $\mathbf{\ddot{x}} = \ddot{x}\mathbf{\hat{i}}$, e escrevemos

\begin{align}
F(t) -  kx - c\dot{x} &= m\ddot{x} \\
\ddot{x} + \frac{c}{m}\dot{x} + \frac{k}{m}x &= F(t)
\end{align}

Essa equação é uma EDO linear, não-homogênea e de segunda ordem. Através do estudo dos movimentos ondulatórios,
sabemos que isso é a equação de um oscilador amortecido e forçado. A solução analítica dessa equação
corresponde à soma de uma solução particular (**estacionária**) com outra solução para a EDO homogêna correspondente (**transiente**).

$$x(t) = x_{hom}(t) + x_{est}(t)$$

### Solução homogênea

![](./oscilacoes_amortecidas.png)

A solução homogênea corresponde a um oscilador amortecido, que se divide em 3 casos: subcrítico, crítico, e supercrítico.
Essa divisão depende dos valores de $\gamma=\frac{c}{m}$ (o *coeficiente de amortecimento*) e de $\omega_0=\sqrt{\frac{k}{m}}$
(a *frequência natural da oscilação*).

- **Caso subcrítico $(\frac{\gamma}{2} < \omega_0)$**

A massa realiza um movimento oscilatório periódico de amplitude decrescente. A solução é

$$x_{hom}(t) = Ae^{-\frac{\gamma}{2}t}\cos(\omega t + \phi)$$

Onde $A$ e $\phi$ dependem das condições iniciais, e $\omega = \sqrt{\omega_0^2 - \frac{\gamma^2}{4}}$

- **Caso crítico $(\frac{\gamma}{2} = \omega_0)$**

A massa rápidamente retorna à posição inicial sem oscilar, parando no ponto de equilíbrio em tempo infinito.

$$x_{hom}(t) = ae^{-\frac{\gamma}{2}t} + bte^{-\frac{\gamma}{2}t}$$

Onde $a$ e $b$ dependem das condições iniciais.

- **Caso supercrítico $(\frac{\gamma}{2} > \omega_0)$**

A massa retorna à posição inicial sem oscilar, mas mais lentamente do que no caso crítico.

$$x_{hom}(t) = ae^{-(\frac{\gamma}{2} - \beta)t} + be^{-(\frac{\gamma}{2} + \beta)t}$$

Onde $a$ e $b$ dependem das condições iniciais, e $\beta=\sqrt{\frac{\gamma^2}{4} - \omega_0^2}$

### Solução estacionária

Dividimos nos possíveis casos para a força externa: nula, constante mas diferente de zero, e periódica.

- $F(t) = 0$

A EDO passa a ser homogênea, e a solução geral é a própria solução homogênea

- $F(t) = F$, com $F=cte$

$$x_{est}(t) = \frac{F}{k}$$

- $F(t) = F_0\cos(\omega t)$

$$x_{est}(t) = A(\omega)\cos[\omega t + \phi(\omega)]$$

Onde

$$A(\omega) = \frac{F_0}{m\sqrt{(\omega_0^2 - \omega^2)^2 + \gamma^2\omega^2}}$$
$$\phi(\omega) = -\arctan\left(\frac{\gamma\,\omega}{\omega_0^2 - \omega^2}\right)$$

## Solução numérica

Para encontrar soluções numéricas, precisamos de um modelo adequado. Para isso, utilizaremos o vetor de estados $\{y\}$, definido como:

$$\{y\} = \begin{pmatrix} x \\ \dot{x} \end{pmatrix} \implies \{\dot{y}\} = \begin{pmatrix} \dot{x} \\ \ddot{x} \end{pmatrix}$$

O que nos dá

$$\{\dot{y}\} = \begin{pmatrix} \dot{x} \\
                                -\frac{k}{m}x - \frac{c}{m}\dot{x} + \frac{F(t)}{m}
                \end{pmatrix} = \begin{pmatrix} 0 & 1 \\ -\frac{k}{m} & -\frac{c}{m} \end{pmatrix} \begin{pmatrix} x \\ \dot{x} \end{pmatrix} + \begin{pmatrix} 0 \\ \frac{F(t)}{m} \end{pmatrix}$$
                
Em Python, isso nos dá:

In [1]:


# EDO: retorna a derivada do vetor de estados dy/dt em funcao do estado atual y e do tempo t
# Note que F e' uma funcao, e c a constante do amortecedor
def massa_mola_amort(y, t, F, c):
    x, x_dot = y # position and velocity
    force = F(t)
    dydt = [v, -k/m*x -c/m*x_dot + force/m]
    return dydt

Também precisamos de uma função para desenhar os gráficos necessários para a análise

In [2]:
import matplotlib.pyplot as plt

def draw_plots(sol, t, F, title):
    # x(t) * t
    plt.subplot(2, 2, 1)
    plt.plot(t, sol[:, 0], 'b')
    plt.xlabel('t (s)')
    plt.ylabel('x (m)')
    plt.grid()

    # v(t) * t
    plt.subplot(2, 2, 2)
    plt.plot(t, sol[:, 1], 'g')
    plt.xlabel('t (s)')
    plt.ylabel('v (m/s)')
    plt.grid()

    # x(t) * v(t)
    plt.subplot(2, 2, 3)
    plt.plot(sol[:, 1], sol[:, 0], 'r')
    plt.xlabel('v (m/s)')
    plt.ylabel('x (m)')
    plt.grid()

    # F(t) * t
    plt.subplot(2, 2, 4)
    plt.plot(t, [F(t0) for t0 in t], 'y')
    plt.xlabel('t (s)')
    plt.ylabel('F (N)')
    plt.grid()

    plt.suptitle(title)
    plt.show()

ModuleNotFoundError: No module named 'matplotlib'

Os casos que serão analisados em seguida possuem variação nas condições iniciais $x(0)$ e $\dot{x}(0)$, na força externa $F(t)$, no período considerado $\Delta t$ e na constante do amortecedor $c$. Definimos as constantes do sistema, assim como uma função para a força externa nula:

In [ ]:
# Bibliotecas/funcoes necessarias
import numpy as np
from scipy.integrate import odeint

# Constantes do problema
m = 10.0
k = 3553.0
c = 37.7
g = 9.81

# F(t) = 0.0
def F_zero(t):
    return 0

### Caso A:
\begin{align}
    &x(0) = 0.1 \\
    &\dot{x}(0) = 0.0 \\
    &F(t) = 0.0 \\
    &c = 37.7 \\
    &\Delta t = 3
\end{align}

In [ ]:
# Condicoes iniciais
t = np.linspace(0, 3, 100)
x0 = 0.10
x_dot0 = 0.0
F = F_zero
# Resolx_dote a EDO
solA = odeint(massa_mola_amort, [x0, x_dot0], t, args=(F, c))
# Graficos
draw_plots(solA, t, F, "Caso A")

### Caso B:
\begin{align}
    &x(0) = 0.0 \\
    &\dot{x}(0) = 1.0 \\
    &F(t) = 0.0 \\
    &c = 37.7 \\
    &\Delta t = 3
\end{align}

In [ ]:
# Condicoes iniciais
t = np.linspace(0, 3, 100)
x0 = 0.0
x_dot0 = 1.0
F = F_zero
# Resolve a EDO
solB = odeint(massa_mola_amort, [x0, x_dot0], t, args=(F, c))
# Graficos
draw_plots(solB, t, F, "Caso B")

### Caso C:
\begin{align}
    &x(0) = 0.1 \\
    &\dot{x}(0) = 1.0 \\
    &F(t) = 0.0 \\
    &c = 377 \\
    &\Delta t = 3
\end{align}

In [ ]:
# Condicoes iniciais
t = np.linspace(0, 3, 100)
x0 = 0.10
x_dot0 = 1.0
F = F_zero
c = 377
# Resolve a EDO
solC = odeint(massa_mola_amort, [x0, x_dot0], t, args=(F, c))
# Graficos
draw_plots(solC, t, F, "Caso C")

### Caso D:
\begin{align}
    &x(0) = 0.0 \\
    &\dot{x}(0) = 0.0 \\
    &F(t) = 1000\sin\left(\pi t + \frac{\pi}{2}\right) \\
    &c = 37.7 \\
    &\Delta t = 5
\end{align}

In [ ]:
def F_D(t):
    F0 = 1000
    omega = np.pi
    theta = np.pi/2
    return F0*np.sin(omega*t + theta)
# Condicoes iniciais
t = np.linspace(0, 5, 100)
x0 = 0.0
x_dot0 = 0.0
F = F_D
c = 37.7
# Resolve a EDO
solD = odeint(massa_mola_amort, [x0, x_dot0], t, args=(F, c))
# Graficos
draw_plots(solD, t, F, "Caso D")

### Caso E:
\begin{align}
    &x(0) = 0.0 \\
    &\dot{x}(0) = 0.0 \\
    &F(t) =
      \begin{cases}
        0       & \quad \text{se } t < 0.5 \\
        1000  & \quad \text{se } t >= 0.5
      \end{cases}
    \\
    &c = 37.7 \\
    &\Delta t = 5
\end{align}

In [ ]:
def F_E(t):
    return 1000 if t >= 0.5 else 0
# Condicoes iniciais
t = np.linspace(0, 5, 100)
x0 = 0.0
x_dot0 = 0.0
F = F_E
c = 37.7
# Resolve a EDO
solE = odeint(massa_mola_amort, [x0, x_dot0], t, args=(F, c))
# Graficos
draw_plots(solE, t, F, "Caso E")